In [161]:
import pandas as pd
from unidecode import unidecode
import numpy as np
import os
import seaborn as sns

### Objetivo
Utilizando a classificação ideológica dos partidos, criada no caderno anterior, crio um mapa de calor do poder dos partidos a partir dos resultados das eleições municipais de 2016 e 2020. O peso de cada município será dado pelo PIB e pelo eleitorado.

### Tratamento prévio: codificação dos municípios
Infelizmente o TSE usa uma codificação própria, diferente da usada pelo IBGE.

In [162]:
df_eleitorado = pd.read_csv('../dados_tse/2020/perfil_eleitorado_2020.csv', sep = ';', encoding = 'latin1')


In [163]:
df_eleitorado.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,CD_MUN_SIT_BIOMETRIA,DS_MUN_SIT_BIOMETRIA,NR_ZONA,CD_GENERO,...,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
0,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,6,ENSINO MÉDIO COMPLETO,17,1,1,0
1,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,7,SUPERIOR INCOMPLETO,1,0,0,0
2,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,8,SUPERIOR COMPLETO,8,0,0,0
3,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9599,95 a 99 anos,1,ANALFABETO,28,0,0,0
4,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9599,95 a 99 anos,2,LÊ E ESCREVE,149,0,0,0


In [164]:
df_eleitorado.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'CD_MUN_SIT_BIOMETRIA', 'DS_MUN_SIT_BIOMETRIA',
       'NR_ZONA', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'QT_ELEITORES_PERFIL',
       'QT_ELEITORES_BIOMETRIA', 'QT_ELEITORES_DEFICIENCIA',
       'QT_ELEITORES_INC_NM_SOCIAL'],
      dtype='object')

In [165]:
tab_municipios_tse = df_eleitorado[['CD_MUNICIPIO', 'NM_MUNICIPIO', 'SG_UF']].drop_duplicates().reset_index(drop = True)

In [166]:
tab_municipios_tse.head()

,CD_MUNICIPIO,NM_MUNICIPIO,SG_UF
0,71072,SÃO PAULO,SP
1,86045,AMARAL FERRADOR,RS
2,24074,FEIRA NOVA,PE
3,71714,TAPIRAÍ,SP
4,71730,TAPIRATIBA,SP


In [167]:
len(tab_municipios_tse)

5568

In [168]:
tab_municipios_tse['NM_MUNICIPIO'] = tab_municipios_tse['NM_MUNICIPIO'].apply(lambda x: unidecode(x))

In [169]:
tab_municipios_tse.head()

,CD_MUNICIPIO,NM_MUNICIPIO,SG_UF
0,71072,SAO PAULO,SP
1,86045,AMARAL FERRADOR,RS
2,24074,FEIRA NOVA,PE
3,71714,TAPIRAI,SP
4,71730,TAPIRATIBA,SP


In [170]:
# Tabela do IBGE baixada de https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9088-produto-interno-bruto-dos-municipios.html?=&t=resultados
df_municipios_ibge = pd.read_excel('PIB dos Munic°pios - base de dados 2010-2018.xls', encoding = 'latin1')

In [171]:
df_municipios_ibge.head()

,Ano,Código da Grande Região,Nome da Grande Região,Código da Unidade da Federação,Sigla da Unidade da Federação,Nome da Unidade da Federação,Código do Município,Nome do Município,Região Metropolitana,Código da Mesorregião,...,"Valor adicionado bruto da Indústria, a preços correntes (R$ 1.000)","Valor adicionado bruto dos Serviços, a preços correntes - exceto Administração, defesa, educação e saúde públicas e seguridade social (R$ 1.000)","Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, a preços correntes (R$ 1.000)","Valor adicionado bruto total, a preços correntes (R$ 1.000)","Impostos, líquidos de subsídios, sobre produtos, a preços correntes (R$ 1.000)","Produto Interno Bruto, a preços correntes (R$ 1.000)","Produto Interno Bruto per capita, a preços correntes (R$ 1,00)",Atividade com maior valor adicionado bruto,Atividade com segundo maior valor adicionado bruto,Atividade com terceiro maior valor adicionado bruto
0,2010,1,Norte,11,RO,Rondônia,1100015,Alta Floresta D'Oeste,NaN,1102,...,16118.534,62496.185,93244.656,241119.767,20957.111,262076.878,10731.18,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
1,2010,1,Norte,11,RO,Rondônia,1100023,Ariquemes,NaN,1102,...,287138.585,494946.267,343867.731,1199664.227,165029.553,1364693.780,15103.86,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
2,2010,1,Norte,11,RO,Rondônia,1100031,Cabixi,NaN,1102,...,3252.506,12677.210,25170.235,65400.772,4210.342,69611.114,11033.62,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
3,2010,1,Norte,11,RO,Rondônia,1100049,Cacoal,NaN,1102,...,182051.537,465447.325,298454.309,1041212.374,145281.717,1186494.091,15095.15,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
4,2010,1,Norte,11,RO,Rondônia,1100056,Cerejeiras,NaN,1102,...,19734.484,80724.991,63018.270,192454.160,29567.029,222021.189,13037.06,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...


In [172]:
df_municipios_ibge.columns

Index(['Ano', 'Código da Grande Região', 'Nome da Grande Região',
       'Código da Unidade da Federação', 'Sigla da Unidade da Federação',
       'Nome da Unidade da Federação', 'Código do Município',
       'Nome do Município', 'Região Metropolitana', 'Código da Mesorregião',
       'Nome da Mesorregião', 'Código da Microrregião', 'Nome da Microrregião',
       'Código da Região Geográfica Imediata',
       'Nome da Região Geográfica Imediata',
       'Município da Região Geográfica Imediata',
       'Código da Região Geográfica Intermediária',
       'Nome da Região Geográfica Intermediária',
       'Município da Região Geográfica Intermediária',
       'Código Concentração Urbana', 'Nome Concentração Urbana',
       'Tipo Concentração Urbana', 'Código Arranjo Populacional',
       'Nome Arranjo Populacional', 'Hierarquia Urbana',
       'Hierarquia Urbana (principais categorias)', 'Código da Região Rural',
       'Nome da Região Rural',
       'Região rural (segundo classificação d

In [173]:
tab_municipios_ibge = df_municipios_ibge[['Código do Município', 'Nome do Município', 'Sigla da Unidade da Federação']].drop_duplicates().reset_index(drop = True)
tab_municipios_ibge.columns = ['CD_MUNICIPIO_IBGE', 'NM_MUNICIPIO', 'SG_UF']

In [174]:
tab_municipios_ibge.head()

,CD_MUNICIPIO_IBGE,NM_MUNICIPIO,SG_UF
0,1100015,Alta Floresta D'Oeste,RO
1,1100023,Ariquemes,RO
2,1100031,Cabixi,RO
3,1100049,Cacoal,RO
4,1100056,Cerejeiras,RO


In [175]:
tab_municipios_ibge['NM_MUNICIPIO'] = tab_municipios_ibge['NM_MUNICIPIO'].apply(lambda x: unidecode(x).upper())

In [176]:
tab_municipios_ibge.head()

,CD_MUNICIPIO_IBGE,NM_MUNICIPIO,SG_UF
0,1100015,ALTA FLORESTA D'OESTE,RO
1,1100023,ARIQUEMES,RO
2,1100031,CABIXI,RO
3,1100049,CACOAL,RO
4,1100056,CEREJEIRAS,RO


In [177]:
# Complementando a tabela do TSE com o código de município do IBGE
tab_municipios_tse = tab_municipios_tse.merge(tab_municipios_ibge, how = 'left', on = ['NM_MUNICIPIO', 'SG_UF'])

In [178]:
tab_municipios_tse.head()

,CD_MUNICIPIO,NM_MUNICIPIO,SG_UF,CD_MUNICIPIO_IBGE
0,71072,SAO PAULO,SP,3550308.0
1,86045,AMARAL FERRADOR,RS,4300638.0
2,24074,FEIRA NOVA,PE,2605400.0
3,71714,TAPIRAI,SP,3553500.0
4,71730,TAPIRATIBA,SP,3553609.0


In [179]:
# Investigando casos em que não funcionou o merge (Ufa! Foram apenas 15 casos!)
tab_municipios_tse[pd.isna(tab_municipios_tse['CD_MUNICIPIO_IBGE']) == True]

,CD_MUNICIPIO,NM_MUNICIPIO,SG_UF,CD_MUNICIPIO_IBGE
911,73199,TABOCAO,TO,NaN
1083,81191,GRAO-PARA,SC,NaN
1129,16233,AREZ,RN,NaN
1510,40827,SEM PEIXE,MG,NaN
1633,71013,SAO LUIS DO PARAITINGA,SP,NaN
1650,16039,ASSU,RN,NaN
2481,5290,SANTA ISABEL DO PARA,PA,NaN
2718,337,ALVORADA DO OESTE,RO,NaN
3673,17035,BOA SAUDE,RN,NaN
4102,31518,GRACCHO CARDOSO,SE,NaN


In [180]:
# Agora vou editando o dataframe na mão, usando esse link aqui pra pesquisar: https://www.ibge.gov.br/explica/codigos-dos-municipios.php

tab_municipios_tse.at[911, 'CD_MUNICIPIO_IBGE'] = 1708254
tab_municipios_tse.at[1083, 'CD_MUNICIPIO_IBGE'] = 4206108
tab_municipios_tse.at[1129, 'CD_MUNICIPIO_IBGE'] = 2401206
tab_municipios_tse.at[1510, 'CD_MUNICIPIO_IBGE'] = 3165560
tab_municipios_tse.at[1633, 'CD_MUNICIPIO_IBGE'] = 3550001
tab_municipios_tse.at[1650, 'CD_MUNICIPIO_IBGE'] = 2400208
tab_municipios_tse.at[2481, 'CD_MUNICIPIO_IBGE'] = 1506500
tab_municipios_tse.at[2718, 'CD_MUNICIPIO_IBGE'] = 1100346
tab_municipios_tse.at[3673, 'CD_MUNICIPIO_IBGE'] = 2405306
tab_municipios_tse.at[4102, 'CD_MUNICIPIO_IBGE'] = 2802601
tab_municipios_tse.at[4753, 'CD_MUNICIPIO_IBGE'] = 1100098
tab_municipios_tse.at[4848, 'CD_MUNICIPIO_IBGE'] = 2905602
tab_municipios_tse.at[5160, 'CD_MUNICIPIO_IBGE'] = 1502954
tab_municipios_tse.at[5174, 'CD_MUNICIPIO_IBGE'] = 2401305
tab_municipios_tse.at[5332, 'CD_MUNICIPIO_IBGE'] = 3145455



In [181]:
tab_municipios_tse[pd.isna(tab_municipios_tse['CD_MUNICIPIO_IBGE']) == True]

,CD_MUNICIPIO,NM_MUNICIPIO,SG_UF,CD_MUNICIPIO_IBGE


### 1. Mapa de poder - eleições de 2020

In [182]:
df_votos_20 = pd.read_csv('../dados_tse/2020/votacao_candidato_munzona_2020_BRASIL.csv', sep = ';', encoding = 'latin1')


In [183]:
df_votos_20.head()


,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_VOTO_EM_TRANSITO,QT_VOTOS_NOMINAIS
0,17/01/2021,19:06:41,2020,2,Eleição Ordinária,1,445,ELEIÇÕES MUNICIPAIS 2020 - AP,06/12/2020,M,...,12,PDT,Partido Democrático Trabalhista,30000098451,PARTIDO ISOLADO,PDT,5,SUPLENTE,N,197
1,17/01/2021,19:06:41,2020,2,Eleição Ordinária,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,...,17,PSL,Partido Social Liberal,240000056242,PARTIDO ISOLADO,PSL,5,SUPLENTE,N,93
2,17/01/2021,19:06:41,2020,2,Eleição Ordinária,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,...,11,PP,PROGRESSISTAS,240000151647,PARTIDO ISOLADO,PP,5,SUPLENTE,N,49
3,17/01/2021,19:06:41,2020,2,Eleição Ordinária,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,...,90,PROS,Partido Republicano da Ordem Social,190000110993,PARTIDO ISOLADO,PROS,5,SUPLENTE,N,1
4,17/01/2021,19:06:41,2020,2,Eleição Ordinária,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,...,17,PSL,Partido Social Liberal,130000122633,PARTIDO ISOLADO,PSL,5,SUPLENTE,N,34


In [184]:
df_votos_20.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO', 'ST_VOTO_EM_TRANSITO',
       'QT_VOTOS_NOMINAIS'],
      dtype='object')

In [185]:
set(df_votos_20['DS_SIT_TOT_TURNO'])

{'2º TURNO',
 'ELEITO',
 'ELEITO POR MÉDIA',
 'ELEITO POR QP',
 'NÃO ELEITO',
 'SUPLENTE'}

In [186]:
len(set(df_votos_20['CD_MUNICIPIO']))

5563

In [187]:
df_prefeituras_20 = df_votos_20[(df_votos_20['DS_SIT_TOT_TURNO'] == 'ELEITO') & (df_votos_20['DS_CARGO'] == 'Prefeito')]


In [188]:
df_prefeituras_resumo_20 = df_prefeituras_20[['CD_MUNICIPIO', 'NM_UE', 'SG_UF', 'SG_PARTIDO', 'QT_VOTOS_NOMINAIS']].groupby(['CD_MUNICIPIO', 'NM_UE', 'SG_UF', 'SG_PARTIDO']).sum().reset_index().sort_values('SG_PARTIDO', ascending = False)
df_prefeituras_resumo_20['ano'] = 2020


In [189]:
df_prefeituras_resumo_20.head()

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano
212,4766,CACHOEIRA DO PIRIÁ,PA,SOLIDARIEDADE,6121,2020
5420,96938,ÁGUA FRIA DE GOIÁS,GO,SOLIDARIEDADE,1820,2020
4938,89826,NOVA SANTA HELENA,MT,SOLIDARIEDADE,1291,2020
1400,24910,OLINDA,PE,SOLIDARIEDADE,123534,2020
778,13471,BOA VIAGEM,CE,SOLIDARIEDADE,19507,2020


In [190]:
len(df_prefeituras_resumo_20)

5478

In [191]:
def corrige_sigla_partido(x):
    if x == 'PC do B':
        y = 'PCdoB'
    elif x == 'PODE':
        y = 'PODEMOS' 
    else:
        y = x
    return y

df_prefeituras_resumo_20['SG_PARTIDO'] = df_prefeituras_resumo_20['SG_PARTIDO'].map(corrige_sigla_partido)

In [192]:
# Ranking por partido
df_prefeituras_resumo_20[['SG_PARTIDO', 'SG_UF']].groupby(['SG_PARTIDO']).count().reset_index().sort_values('SG_UF', ascending = False)


,SG_PARTIDO,SG_UF
4,MDB,785
13,PP,687
18,PSD,654
19,PSDB,523
3,DEM,469
9,PL,346
8,PDT,314
16,PSB,252
23,PTB,215
27,REPUBLICANOS,212


##### -> Validação: todos os municípios estão com resultado? Há municípios duplicados?

In [193]:
df_teste = df_prefeituras_resumo_20[['CD_MUNICIPIO', 'NM_UE', 'SG_UF', 'SG_PARTIDO']].groupby(['CD_MUNICIPIO', 'NM_UE', 'SG_UF']).count()
df_teste[df_teste['SG_PARTIDO'] > 1]


,,,SG_PARTIDO
CD_MUNICIPIO,NM_UE,SG_UF,


In [194]:
df_eleitorado.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,CD_MUN_SIT_BIOMETRIA,DS_MUN_SIT_BIOMETRIA,NR_ZONA,CD_GENERO,...,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
0,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,6,ENSINO MÉDIO COMPLETO,17,1,1,0
1,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,7,SUPERIOR INCOMPLETO,1,0,0,0
2,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,8,SUPERIOR COMPLETO,8,0,0,0
3,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9599,95 a 99 anos,1,ANALFABETO,28,0,0,0
4,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9599,95 a 99 anos,2,LÊ E ESCREVE,149,0,0,0


In [195]:
df_eleitorado.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'CD_MUN_SIT_BIOMETRIA', 'DS_MUN_SIT_BIOMETRIA',
       'NR_ZONA', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'QT_ELEITORES_PERFIL',
       'QT_ELEITORES_BIOMETRIA', 'QT_ELEITORES_DEFICIENCIA',
       'QT_ELEITORES_INC_NM_SOCIAL'],
      dtype='object')

In [196]:
len(set(df_eleitorado['CD_MUNICIPIO']))

5568

In [197]:
df_eleitorado_votos_20 = df_eleitorado[['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'QT_ELEITORES_PERFIL']].groupby(['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO']).sum().reset_index()\
.merge(df_votos_20[df_votos_20['DS_CARGO'] == 'Prefeito'][['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'DS_CARGO', 'QT_VOTOS_NOMINAIS']].groupby(['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'DS_CARGO']).sum(), how = 'left', on = ['SG_UF', 'CD_MUNICIPIO'])


In [198]:
df_eleitorado_votos_20.head()

,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,QT_ELEITORES_PERFIL,QT_VOTOS_NOMINAIS
0,AC,1007,BUJARI,9613,7051.0
1,AC,1015,CAPIXABA,7540,5897.0
2,AC,1023,PORTO ACRE,12639,9531.0
3,AC,1031,SANTA ROSA DO PURUS,3566,2845.0
4,AC,1040,MARECHAL THAUMATURGO,10491,8373.0


In [199]:
len(df_eleitorado_votos_20[pd.isna(df_eleitorado_votos_20['QT_VOTOS_NOMINAIS']) == True])

30

In [200]:
print('Conclusão:')
print('Duplicados = ' + str(len(df_teste[df_teste['SG_PARTIDO'] > 1])))
print('Sem prefeito informado = ' + str(len(set(df_eleitorado['CD_MUNICIPIO'])) - len(set(df_prefeituras_resumo_20['CD_MUNICIPIO']))))

Conclusão:
Duplicados = 0
Sem prefeito informado = 90


##### -> Análise: PIB municipal e Eleitorado

In [201]:
df_prefeituras_resumo_20.head()

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano
212,4766,CACHOEIRA DO PIRIÁ,PA,SOLIDARIEDADE,6121,2020
5420,96938,ÁGUA FRIA DE GOIÁS,GO,SOLIDARIEDADE,1820,2020
4938,89826,NOVA SANTA HELENA,MT,SOLIDARIEDADE,1291,2020
1400,24910,OLINDA,PE,SOLIDARIEDADE,123534,2020
778,13471,BOA VIAGEM,CE,SOLIDARIEDADE,19507,2020


In [202]:
df_prefeituras_resumo_20 = df_prefeituras_resumo_20.merge(tab_municipios_tse[['CD_MUNICIPIO', 'SG_UF', 'CD_MUNICIPIO_IBGE']], how = 'inner', on = ['CD_MUNICIPIO', 'SG_UF'])


In [203]:
df_prefeituras_resumo_20.head()

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE
0,4766,CACHOEIRA DO PIRIÁ,PA,SOLIDARIEDADE,6121,2020,1501956.0
1,96938,ÁGUA FRIA DE GOIÁS,GO,SOLIDARIEDADE,1820,2020,5200175.0
2,89826,NOVA SANTA HELENA,MT,SOLIDARIEDADE,1291,2020,5106190.0
3,24910,OLINDA,PE,SOLIDARIEDADE,123534,2020,2609600.0
4,13471,BOA VIAGEM,CE,SOLIDARIEDADE,19507,2020,2302404.0


In [204]:
# Tabela do IBGE baixada de https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9088-produto-interno-bruto-dos-municipios.html?=&t=resultados
df_municipios_ibge = pd.read_excel('PIB dos Munic°pios - base de dados 2010-2018.xls', encoding = 'latin1')


In [205]:
df_municipios_ibge.columns

Index(['Ano', 'Código da Grande Região', 'Nome da Grande Região',
       'Código da Unidade da Federação', 'Sigla da Unidade da Federação',
       'Nome da Unidade da Federação', 'Código do Município',
       'Nome do Município', 'Região Metropolitana', 'Código da Mesorregião',
       'Nome da Mesorregião', 'Código da Microrregião', 'Nome da Microrregião',
       'Código da Região Geográfica Imediata',
       'Nome da Região Geográfica Imediata',
       'Município da Região Geográfica Imediata',
       'Código da Região Geográfica Intermediária',
       'Nome da Região Geográfica Intermediária',
       'Município da Região Geográfica Intermediária',
       'Código Concentração Urbana', 'Nome Concentração Urbana',
       'Tipo Concentração Urbana', 'Código Arranjo Populacional',
       'Nome Arranjo Populacional', 'Hierarquia Urbana',
       'Hierarquia Urbana (principais categorias)', 'Código da Região Rural',
       'Nome da Região Rural',
       'Região rural (segundo classificação d

In [206]:
df_municipios_ibge[df_municipios_ibge['Ano'] == 2018][['Código do Município', 'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)']].head()


,Código do Município,"Produto Interno Bruto, a preços correntes (R$ 1.000)"
44545,1100015,499305.982
44546,1100023,2463773.260
44547,1100031,140502.269
44548,1100049,2180454.016
44549,1100056,472178.834


In [207]:
df_prefeituras_resumo_20 = df_prefeituras_resumo_20.merge(df_municipios_ibge[df_municipios_ibge['Ano'] == 2018][['Código do Município', 'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)']], how = 'inner', left_on = 'CD_MUNICIPIO_IBGE', right_on = 'Código do Município')


In [208]:
df_prefeituras_resumo_20.columns = ['CD_MUNICIPIO', 'NM_UE', 'SG_UF', 'SG_PARTIDO', 'QT_VOTOS_NOMINAIS', 'ano', 'CD_MUNICIPIO_IBGE', 'Código do Município', 'PIB a preços correntes (em R$ 1000)']

In [209]:
df_prefeituras_resumo_20

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,Código do Município,PIB a preços correntes (em R$ 1000)
0,4766,CACHOEIRA DO PIRIÁ,PA,SOLIDARIEDADE,6121,2020,1501956.0,1501956,179439.409
1,96938,ÁGUA FRIA DE GOIÁS,GO,SOLIDARIEDADE,1820,2020,5200175.0,5200175,255565.620
2,89826,NOVA SANTA HELENA,MT,SOLIDARIEDADE,1291,2020,5106190.0,5106190,174711.050
3,24910,OLINDA,PE,SOLIDARIEDADE,123534,2020,2609600.0,2609600,5533876.468
4,13471,BOA VIAGEM,CE,SOLIDARIEDADE,19507,2020,2302404.0,2302404,382775.595
5,95419,PIRANHAS,GO,SOLIDARIEDADE,2552,2020,5217203.0,5217203,378442.512
6,65994,JAMBEIRO,SP,SOLIDARIEDADE,1731,2020,3524907.0,3524907,238349.676
7,73180,SÃO MIGUEL DO TOCANTINS,TO,SOLIDARIEDADE,3125,2020,1720200.0,1720200,105746.341
8,42447,UNIÃO DE MINAS,MG,SOLIDARIEDADE,1395,2020,3170438.0,3170438,111083.184
9,61220,ALUMÍNIO,SP,SOLIDARIEDADE,4641,2020,3501152.0,3501152,2105104.507


In [210]:
df_prefeituras_resumo_20 = df_prefeituras_resumo_20.merge(df_eleitorado[['SG_UF', 'CD_MUNICIPIO', 'QT_ELEITORES_PERFIL']].groupby(['SG_UF', 'CD_MUNICIPIO']).sum().reset_index(), how = 'inner', on = ['SG_UF', 'CD_MUNICIPIO'])


In [211]:
df_prefeituras_resumo_20.head()

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,Código do Município,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL
0,4766,CACHOEIRA DO PIRIÁ,PA,SOLIDARIEDADE,6121,2020,1501956.0,1501956,179439.409,14219
1,96938,ÁGUA FRIA DE GOIÁS,GO,SOLIDARIEDADE,1820,2020,5200175.0,5200175,255565.620,4586
2,89826,NOVA SANTA HELENA,MT,SOLIDARIEDADE,1291,2020,5106190.0,5106190,174711.050,2828
3,24910,OLINDA,PE,SOLIDARIEDADE,123534,2020,2609600.0,2609600,5533876.468,282952
4,13471,BOA VIAGEM,CE,SOLIDARIEDADE,19507,2020,2302404.0,2302404,382775.595,39155


In [212]:
len(df_prefeituras_resumo_20)

5478

---*---*---

##### -> Grupos ideológicos

In [213]:
df_resumo_posicionamentos_ = pd.read_pickle('df_resumo_posicionamentos_pickle')

In [217]:
df_prefeituras_resumo_20_ = df_prefeituras_resumo_20.merge(df_resumo_posicionamentos_[['Partido', 'grupo_ideologico']], how = 'inner', left_on = 'SG_PARTIDO', right_on = 'Partido')


In [218]:
len(df_prefeituras_resumo_20_)

5456

In [219]:
df_prefeituras_resumo_20_

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,Código do Município,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL,Partido,grupo_ideologico
0,4766,CACHOEIRA DO PIRIÁ,PA,SOLIDARIEDADE,6121,2020,1501956.0,1501956,179439.409,14219,SOLIDARIEDADE,liberal_conservador
1,96938,ÁGUA FRIA DE GOIÁS,GO,SOLIDARIEDADE,1820,2020,5200175.0,5200175,255565.620,4586,SOLIDARIEDADE,liberal_conservador
2,89826,NOVA SANTA HELENA,MT,SOLIDARIEDADE,1291,2020,5106190.0,5106190,174711.050,2828,SOLIDARIEDADE,liberal_conservador
3,24910,OLINDA,PE,SOLIDARIEDADE,123534,2020,2609600.0,2609600,5533876.468,282952,SOLIDARIEDADE,liberal_conservador
4,13471,BOA VIAGEM,CE,SOLIDARIEDADE,19507,2020,2302404.0,2302404,382775.595,39155,SOLIDARIEDADE,liberal_conservador
5,95419,PIRANHAS,GO,SOLIDARIEDADE,2552,2020,5217203.0,5217203,378442.512,9156,SOLIDARIEDADE,liberal_conservador
6,65994,JAMBEIRO,SP,SOLIDARIEDADE,1731,2020,3524907.0,3524907,238349.676,4859,SOLIDARIEDADE,liberal_conservador
7,73180,SÃO MIGUEL DO TOCANTINS,TO,SOLIDARIEDADE,3125,2020,1720200.0,1720200,105746.341,7226,SOLIDARIEDADE,liberal_conservador
8,42447,UNIÃO DE MINAS,MG,SOLIDARIEDADE,1395,2020,3170438.0,3170438,111083.184,2958,SOLIDARIEDADE,liberal_conservador
9,61220,ALUMÍNIO,SP,SOLIDARIEDADE,4641,2020,3501152.0,3501152,2105104.507,14276,SOLIDARIEDADE,liberal_conservador


In [220]:
df_prefeituras_resumo_20_['NM_UE_UF'] = df_prefeituras_resumo_20_['NM_UE'] + '-' + df_prefeituras_resumo_20_['SG_UF']

In [221]:
df_prefeituras_resumo_20_[['NM_UE_UF', 'grupo_ideologico', 'SG_PARTIDO', 'QT_ELEITORES_PERFIL', 'PIB a preços correntes (em R$ 1000)']].sample(5)


,NM_UE_UF,grupo_ideologico,SG_PARTIDO,QT_ELEITORES_PERFIL,PIB a preços correntes (em R$ 1000)
5360,CASCAVEL-CE,liberal_conservador,CIDADANIA,54723,7.376780e+05
5228,SANTO AFONSO-MT,liberal_conservador,DEM,2303,7.596663e+04
2967,ITAPORANGA D AJUDA-SE,liberal_conservador,PP,27201,9.036046e+05
1045,TABULEIRO-MG,liberal_progressista,PSDB,4248,5.047880e+04
430,IPOJUCA-PE,liberal_conservador,PTB,76990,1.157055e+07


In [222]:
df_prefeituras_resumo_20_[['NM_UE_UF', 'grupo_ideologico', 'SG_PARTIDO', 'QT_ELEITORES_PERFIL', 'PIB a preços correntes (em R$ 1000)']]\
.to_excel('df_prefeituras_resumo_20.xlsx', encoding = 'latin1')

### 2. Mapa de poder - eleições de 2016

In [223]:
df_votos_16 = pd.read_csv('../dados_tse/2016/votacao_candidato_munzona_2016_BRASIL.csv', sep = ';', encoding = 'latin1')


In [224]:
df_votos_16.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_VOTO_EM_TRANSITO,QT_VOTOS_NOMINAIS
0,21/02/2020,12:42:49,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,45,PSDB,Partido da Social Democracia Brasileira,120000000293,UNIDOS POR MIRANDA,PSL / PR / DEM / PRP / PSDB / PV,4,NÃO ELEITO,N,5347
1,21/02/2020,12:42:49,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,33,PMN,Partido da Mobilização Nacional,120000000488,UNIÃO POR MIRANDA II,PHS / PMN,4,NÃO ELEITO,N,12
2,21/02/2020,12:42:49,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,31,PHS,Partido Humanista da Solidariedade,120000000488,UNIÃO POR MIRANDA II,PHS / PMN,4,NÃO ELEITO,N,54
3,21/02/2020,12:42:49,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,45,PSDB,Partido da Social Democracia Brasileira,120000000297,UNIDOS POR MIRANDA I,PSDB / DEM / PRP,4,NÃO ELEITO,N,0
4,21/02/2020,12:42:49,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,12,PDT,Partido Democrático Trabalhista,130000002817,A VOZ DO POVO,PDT / PTB / PMDB / PMN,5,SUPLENTE,N,18


In [225]:
df_votos_16.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO', 'ST_VOTO_EM_TRANSITO',
       'QT_VOTOS_NOMINAIS'],
      dtype='object')

In [226]:
set(df_votos_16['DS_SIT_TOT_TURNO'])

{'2º TURNO',
 'ELEITO',
 'ELEITO POR MÉDIA',
 'ELEITO POR QP',
 'NÃO ELEITO',
 'SUPLENTE'}

In [227]:
set(df_votos_16['NM_TIPO_ELEICAO'])

{'Eleição Ordinária', 'Eleição Suplementar'}

In [228]:
len(set(df_votos_16['CD_MUNICIPIO']))

5567

In [229]:
df_prefeituras_16 = df_votos_16[(df_votos_16['DS_SIT_TOT_TURNO'] == 'ELEITO') &\
                                (df_votos_16['DS_CARGO'] == 'Prefeito') &\
                                (df_votos_16['NM_TIPO_ELEICAO'] == 'Eleição Ordinária')]


In [230]:
df_prefeituras_resumo_16 = df_prefeituras_16[['CD_MUNICIPIO', 'NM_UE', 'SG_UF', 'SG_PARTIDO', 'QT_VOTOS_NOMINAIS']].groupby(['CD_MUNICIPIO', 'NM_UE', 'SG_UF', 'SG_PARTIDO']).sum().reset_index().sort_values('SG_PARTIDO', ascending = False)
df_prefeituras_resumo_16['ano'] = 2016


In [231]:
df_prefeituras_resumo_16.head()

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano
2553,46973,JACUTINGA,MG,SD,7753,2016
1337,23256,BARRA DE GUABIRABA,PE,SD,4210,2016
1637,31399,DIVINA PASTORA,SE,SD,1430,2016
1627,31151,BOQUIM,SE,SD,9612,2016
757,13153,ANTONINA DO NORTE,CE,SD,3225,2016


In [232]:
len(df_prefeituras_resumo_16)

5520

In [233]:
# Ranking por partido
df_prefeituras_resumo_16[['SG_PARTIDO', 'SG_UF']].groupby(['SG_PARTIDO']).count().reset_index().sort_values('SG_UF', ascending = False).head(10)


,SG_PARTIDO,SG_UF
6,PMDB,1044
19,PSDB,799
18,PSD,538
8,PP,495
16,PSB,407
2,PDT,334
11,PR,297
0,DEM,268
25,PTB,256
23,PT,254


##### -> Validação: todos os municípios estão com resultado? Municípios duplicados?

In [234]:
len(df_prefeituras_resumo_16)

5520

In [235]:
df_teste = df_prefeituras_resumo_16[['CD_MUNICIPIO', 'NM_UE', 'SG_UF', 'SG_PARTIDO']].groupby(['CD_MUNICIPIO', 'NM_UE', 'SG_UF']).agg({'SG_PARTIDO': pd.Series.nunique})
df_teste[df_teste['SG_PARTIDO'] > 1]


,,,SG_PARTIDO
CD_MUNICIPIO,NM_UE,SG_UF,


In [236]:
df_eleitorado.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,CD_MUN_SIT_BIOMETRIA,DS_MUN_SIT_BIOMETRIA,NR_ZONA,CD_GENERO,...,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
0,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,6,ENSINO MÉDIO COMPLETO,17,1,1,0
1,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,7,SUPERIOR INCOMPLETO,1,0,0,0
2,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,8,SUPERIOR COMPLETO,8,0,0,0
3,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9599,95 a 99 anos,1,ANALFABETO,28,0,0,0
4,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9599,95 a 99 anos,2,LÊ E ESCREVE,149,0,0,0


In [237]:
df_eleitorado.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'CD_MUN_SIT_BIOMETRIA', 'DS_MUN_SIT_BIOMETRIA',
       'NR_ZONA', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'QT_ELEITORES_PERFIL',
       'QT_ELEITORES_BIOMETRIA', 'QT_ELEITORES_DEFICIENCIA',
       'QT_ELEITORES_INC_NM_SOCIAL'],
      dtype='object')

In [238]:
len(set(df_eleitorado['CD_MUNICIPIO']))

5568

In [239]:
df_eleitorado_votos_16 = df_eleitorado[['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'QT_ELEITORES_PERFIL']].groupby(['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO']).sum().reset_index()\
.merge(df_votos_16[df_votos_16['DS_CARGO'] == 'Prefeito'][['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'DS_CARGO', 'QT_VOTOS_NOMINAIS']].groupby(['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'DS_CARGO']).sum(), how = 'left', on = ['SG_UF', 'CD_MUNICIPIO'])


In [240]:
df_eleitorado_votos_16.head()

,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,QT_ELEITORES_PERFIL,QT_VOTOS_NOMINAIS
0,AC,1007,BUJARI,9613,6204.0
1,AC,1015,CAPIXABA,7540,6155.0
2,AC,1023,PORTO ACRE,12639,9793.0
3,AC,1031,SANTA ROSA DO PURUS,3566,2419.0
4,AC,1040,MARECHAL THAUMATURGO,10491,7244.0


In [241]:
df_eleitorado_votos_16[pd.isna(df_eleitorado_votos_16['QT_VOTOS_NOMINAIS']) == True]

,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,QT_ELEITORES_PERFIL,QT_VOTOS_NOMINAIS
1304,MA,8931,SAMBAÍBA,5569,NaN
4666,SC,83194,SÃO FRANCISCO DO SUL,37386,NaN


In [242]:
len(df_eleitorado_votos_16[pd.isna(df_eleitorado_votos_16['QT_VOTOS_NOMINAIS']) == True])

2

In [243]:
df_prefeituras_resumo_16 = df_prefeituras_resumo_16.merge(tab_municipios_tse[['CD_MUNICIPIO', 'SG_UF', 'CD_MUNICIPIO_IBGE']], how = 'inner', on = ['CD_MUNICIPIO', 'SG_UF'])


In [244]:
df_prefeituras_resumo_16.head()

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE
0,46973,JACUTINGA,MG,SD,7753,2016,3134905.0
1,23256,BARRA DE GUABIRABA,PE,SD,4210,2016,2601300.0
2,31399,DIVINA PASTORA,SE,SD,1430,2016,2802007.0
3,31151,BOQUIM,SE,SD,9612,2016,2800670.0
4,13153,ANTONINA DO NORTE,CE,SD,3225,2016,2300804.0


In [245]:
len(df_prefeituras_resumo_16)

5520

In [246]:
df_prefeituras_resumo_16 = df_prefeituras_resumo_16.merge(df_municipios_ibge[df_municipios_ibge['Ano'] == 2018][['Código do Município', 'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)']], how = 'inner', left_on = 'CD_MUNICIPIO_IBGE', right_on = 'Código do Município')


In [247]:
df_prefeituras_resumo_16.columns = ['CD_MUNICIPIO', 'NM_UE', 'SG_UF', 'SG_PARTIDO', 'QT_VOTOS_NOMINAIS', 'ano', 'CD_MUNICIPIO_IBGE', 'Código do Município', 'PIB a preços correntes (em R$ 1000)']


In [248]:
df_prefeituras_resumo_16

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,Código do Município,PIB a preços correntes (em R$ 1000)
0,46973,JACUTINGA,MG,SD,7753,2016,3134905.0,3134905,6.927060e+05
1,23256,BARRA DE GUABIRABA,PE,SD,4210,2016,2601300.0,2601300,1.291933e+05
2,31399,DIVINA PASTORA,SE,SD,1430,2016,2802007.0,2802007,1.526799e+05
3,31151,BOQUIM,SE,SD,9612,2016,2800670.0,2800670,2.923632e+05
4,13153,ANTONINA DO NORTE,CE,SD,3225,2016,2300804.0,2300804,5.342257e+04
5,79898,VERA CRUZ DO OESTE,PR,SD,2233,2016,4128559.0,4128559,2.570005e+05
6,13102,QUITERIANÓPOLIS,CE,SD,6749,2016,2311264.0,2311264,1.451854e+05
7,2178,BORBA,AM,SD,6238,2016,1300805.0,1300805,3.037137e+05
8,13005,JIJOCA DE JERICOACOARA,CE,SD,4084,2016,2307254.0,2307254,3.474881e+05
9,42110,CAMBUÍ,MG,SD,5833,2016,3110608.0,3110608,1.158960e+06


In [249]:
df_prefeituras_resumo_16 = df_prefeituras_resumo_16.merge(df_eleitorado[['SG_UF', 'CD_MUNICIPIO', 'QT_ELEITORES_PERFIL']].groupby(['SG_UF', 'CD_MUNICIPIO']).sum().reset_index(), how = 'inner', on = ['SG_UF', 'CD_MUNICIPIO'])


In [250]:
df_prefeituras_resumo_16.head()

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,Código do Município,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL
0,46973,JACUTINGA,MG,SD,7753,2016,3134905.0,3134905,692706.027,18297
1,23256,BARRA DE GUABIRABA,PE,SD,4210,2016,2601300.0,2601300,129193.317,9466
2,31399,DIVINA PASTORA,SE,SD,1430,2016,2802007.0,2802007,152679.864,3936
3,31151,BOQUIM,SE,SD,9612,2016,2800670.0,2800670,292363.173,20880
4,13153,ANTONINA DO NORTE,CE,SD,3225,2016,2300804.0,2300804,53422.569,7198


In [251]:
len(df_prefeituras_resumo_16)

5520

In [252]:
print('Conclusão:')
print('Duplicados = ' + str(len(df_teste[df_teste['SG_PARTIDO'] > 1])))
print('Sem prefeito informado = ' + str(len(set(df_eleitorado['CD_MUNICIPIO'])) - len(set(df_prefeituras_resumo_16['CD_MUNICIPIO']))))

Conclusão:
Duplicados = 0
Sem prefeito informado = 48


##### -> Grupos ideológicos

In [253]:
# Preciso corrigir as siglas, pois há algumas mudanças entre 2016 e 2018.
def corrige_sigla_partido(x):
    if x == 'PMDB':
        y = 'MDB'
    elif x == 'PR':
        y = 'PL'
    elif x == 'PPS':
        y = 'CIDADANIA'
    elif x == 'PRB':
        y = 'REPUBLICANOS'
    elif x == 'SD':
        y = 'SOLIDARIEDADE'
    elif x == 'PHS':
        y = 'PODEMOS'
    elif x == 'PTN':
        y = 'PODEMOS'
    elif x == 'PRP':
        y = 'PATRIOTA'
    elif x == 'PEN':
        y = 'PATRIOTA'
    elif x == 'PT do B':
        y = 'AVANTE'
    elif x == 'PSDC':
        y = 'DC'
    elif x == 'PPL':
        y = 'PCdoB'
    elif x == 'PC do B':
        y = 'PCdoB'
    elif x == 'PATRI':
        y = 'PATRIOTA'
    else:
        y = x
    return y
    

In [254]:
df_prefeituras_resumo_16_ = df_prefeituras_resumo_16.copy()
df_prefeituras_resumo_16_['SG_PARTIDO'] = df_prefeituras_resumo_16_['SG_PARTIDO'].map(corrige_sigla_partido)
df_prefeituras_resumo_16_ = df_prefeituras_resumo_16_.merge(df_resumo_posicionamentos_[['Partido', 'grupo_ideologico']], how = 'inner', left_on = 'SG_PARTIDO', right_on = 'Partido')


In [255]:
df_prefeituras_resumo_16_

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,Código do Município,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL,Partido,grupo_ideologico
0,46973,JACUTINGA,MG,SOLIDARIEDADE,7753,2016,3134905.0,3134905,6.927060e+05,18297,SOLIDARIEDADE,liberal_conservador
1,23256,BARRA DE GUABIRABA,PE,SOLIDARIEDADE,4210,2016,2601300.0,2601300,1.291933e+05,9466,SOLIDARIEDADE,liberal_conservador
2,31399,DIVINA PASTORA,SE,SOLIDARIEDADE,1430,2016,2802007.0,2802007,1.526799e+05,3936,SOLIDARIEDADE,liberal_conservador
3,31151,BOQUIM,SE,SOLIDARIEDADE,9612,2016,2800670.0,2800670,2.923632e+05,20880,SOLIDARIEDADE,liberal_conservador
4,13153,ANTONINA DO NORTE,CE,SOLIDARIEDADE,3225,2016,2300804.0,2300804,5.342257e+04,7198,SOLIDARIEDADE,liberal_conservador
5,79898,VERA CRUZ DO OESTE,PR,SOLIDARIEDADE,2233,2016,4128559.0,4128559,2.570005e+05,7006,SOLIDARIEDADE,liberal_conservador
6,13102,QUITERIANÓPOLIS,CE,SOLIDARIEDADE,6749,2016,2311264.0,2311264,1.451854e+05,16936,SOLIDARIEDADE,liberal_conservador
7,2178,BORBA,AM,SOLIDARIEDADE,6238,2016,1300805.0,1300805,3.037137e+05,22473,SOLIDARIEDADE,liberal_conservador
8,13005,JIJOCA DE JERICOACOARA,CE,SOLIDARIEDADE,4084,2016,2307254.0,2307254,3.474881e+05,16193,SOLIDARIEDADE,liberal_conservador
9,42110,CAMBUÍ,MG,SOLIDARIEDADE,5833,2016,3110608.0,3110608,1.158960e+06,21372,SOLIDARIEDADE,liberal_conservador


In [256]:
df_prefeituras_resumo_16_['NM_UE_UF'] = df_prefeituras_resumo_16_['NM_UE'] + '-' + df_prefeituras_resumo_16_['SG_UF']

In [257]:
df_prefeituras_resumo_16_[['NM_UE_UF', 'grupo_ideologico', 'SG_PARTIDO', 'QT_ELEITORES_PERFIL', 'PIB a preços correntes (em R$ 1000)']].sample(5)


,NM_UE_UF,grupo_ideologico,SG_PARTIDO,QT_ELEITORES_PERFIL,PIB a preços correntes (em R$ 1000)
5285,PINHALZINHO-SP,liberal_conservador,DEM,11659,262499.356
119,MANFRINÓPOLIS-PR,liberal_progressista,PV,2990,52596.679
1505,ITAGUARU-GO,liberal_progressista,PSDB,5149,77053.290
156,BREJETUBA-ES,liberal_progressista,PV,9793,244587.303
1849,CENTRALINA-MG,liberal_conservador,PSD,7235,251300.166


In [258]:
len(df_prefeituras_resumo_16_)

5454

In [259]:
df_prefeituras_resumo_16_[['SG_PARTIDO', 'grupo_ideologico']].drop_duplicates().sort_values('grupo_ideologico')

,SG_PARTIDO,grupo_ideologico
61,REDE,intervencionista_progressista
3227,PCdoB,intervencionista_progressista
505,PT,intervencionista_progressista
759,PSOL,intervencionista_progressista
4852,PDT,intervencionista_progressista
0,SOLIDARIEDADE,liberal_conservador
3808,MDB,liberal_conservador
3313,PP,liberal_conservador
3105,CIDADANIA,liberal_conservador
2808,PL,liberal_conservador


In [260]:
df_prefeituras_resumo_16_[['NM_UE_UF', 'grupo_ideologico', 'SG_PARTIDO', 'QT_ELEITORES_PERFIL', 'PIB a preços correntes (em R$ 1000)']]\
.to_excel('df_prefeituras_resumo_16.xlsx', encoding = 'latin1')

### 3. Mapa de poder - eleições de 2012

In [261]:
lista_arquivos = os.listdir('../dados_tse/2012/votacao_candidato_munzona_2012/')
lista_arquivos

['LEIAME.pdf',
 'votacao_candidato_munzona_2012_AC.txt',
 'votacao_candidato_munzona_2012_AL.txt',
 'votacao_candidato_munzona_2012_AM.txt',
 'votacao_candidato_munzona_2012_AP.txt',
 'votacao_candidato_munzona_2012_BA.txt',
 'votacao_candidato_munzona_2012_CE.txt',
 'votacao_candidato_munzona_2012_ES.txt',
 'votacao_candidato_munzona_2012_GO.txt',
 'votacao_candidato_munzona_2012_MA.txt',
 'votacao_candidato_munzona_2012_MG.txt',
 'votacao_candidato_munzona_2012_MS.txt',
 'votacao_candidato_munzona_2012_MT.txt',
 'votacao_candidato_munzona_2012_PA.txt',
 'votacao_candidato_munzona_2012_PB.txt',
 'votacao_candidato_munzona_2012_PE.txt',
 'votacao_candidato_munzona_2012_PI.txt',
 'votacao_candidato_munzona_2012_PR.txt',
 'votacao_candidato_munzona_2012_RJ.txt',
 'votacao_candidato_munzona_2012_RN.txt',
 'votacao_candidato_munzona_2012_RO.txt',
 'votacao_candidato_munzona_2012_RR.txt',
 'votacao_candidato_munzona_2012_RS.txt',
 'votacao_candidato_munzona_2012_SC.txt',
 'votacao_candidato

In [262]:
# Votos: Primeiro preciso juntar os arquivos
import os
lista_arquivos = os.listdir('../dados_tse/2012/votacao_candidato_munzona_2012/')
df_votos_12 = pd.read_csv('../dados_tse/2012/votacao_candidato_munzona_2012/' + lista_arquivos[1], sep = ';', encoding = 'latin1')
df_votos_12.columns = ['DATA_GERACAO', 'HORA_GERACAO', 'ANO_ELEICAO', 'NUM_TURNO (*)', 'DESCRICAO_ELEICAO (*)', 'SIGLA_UF', 'COD_UE (*)', 'CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'NUMERO_ZONA (*)', 'CODIGO_CARGO (*)', 'NUMERO_CAND (*)', 'SQ_CANDIDATO (*)', 'NOME_CANDIDATO', 'NOME_URNA_CANDIDATO', 'DESCRICAO_CARGO', 'COD_SIT_CAND_SUPERIOR', 'DESC_SIT_CAND_SUPERIOR', 'CODIGO_SIT_CANDIDATO', 'DESC_SIT_CANDIDATO', 'CODIGO_SIT_CAND_TOT', 'DESC_SIT_CAND_TOT', 'NUMERO_PARTIDO', 'SIGLA_PARTIDO', 'NOME_PARTIDO', 'SEQUENCIAL_LEGENDA', 'NOME_COLIGACAO', 'COMPOSICAO_LEGENDA', 'TOTAL_VOTOS']

for i in lista_arquivos[2:]:
    df_temp = pd.read_csv('../dados_tse/2012/votacao_candidato_munzona_2012/' + i, sep = ';', encoding = 'latin1')
    df_temp.columns = ['DATA_GERACAO', 'HORA_GERACAO', 'ANO_ELEICAO', 'NUM_TURNO (*)', 'DESCRICAO_ELEICAO (*)', 'SIGLA_UF', 'COD_UE (*)', 'CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'NUMERO_ZONA (*)', 'CODIGO_CARGO (*)', 'NUMERO_CAND (*)', 'SQ_CANDIDATO (*)', 'NOME_CANDIDATO', 'NOME_URNA_CANDIDATO', 'DESCRICAO_CARGO', 'COD_SIT_CAND_SUPERIOR', 'DESC_SIT_CAND_SUPERIOR', 'CODIGO_SIT_CANDIDATO', 'DESC_SIT_CANDIDATO', 'CODIGO_SIT_CAND_TOT', 'DESC_SIT_CAND_TOT', 'NUMERO_PARTIDO', 'SIGLA_PARTIDO', 'NOME_PARTIDO', 'SEQUENCIAL_LEGENDA', 'NOME_COLIGACAO', 'COMPOSICAO_LEGENDA', 'TOTAL_VOTOS']
    df_votos_12 = pd.concat([df_votos_12, df_temp])
    print(i)

votacao_candidato_munzona_2012_AL.txt
votacao_candidato_munzona_2012_AM.txt
votacao_candidato_munzona_2012_AP.txt
votacao_candidato_munzona_2012_BA.txt
votacao_candidato_munzona_2012_CE.txt
votacao_candidato_munzona_2012_ES.txt
votacao_candidato_munzona_2012_GO.txt
votacao_candidato_munzona_2012_MA.txt
votacao_candidato_munzona_2012_MG.txt
votacao_candidato_munzona_2012_MS.txt
votacao_candidato_munzona_2012_MT.txt
votacao_candidato_munzona_2012_PA.txt
votacao_candidato_munzona_2012_PB.txt
votacao_candidato_munzona_2012_PE.txt
votacao_candidato_munzona_2012_PI.txt
votacao_candidato_munzona_2012_PR.txt
votacao_candidato_munzona_2012_RJ.txt
votacao_candidato_munzona_2012_RN.txt
votacao_candidato_munzona_2012_RO.txt
votacao_candidato_munzona_2012_RR.txt
votacao_candidato_munzona_2012_RS.txt
votacao_candidato_munzona_2012_SC.txt
votacao_candidato_munzona_2012_SE.txt
votacao_candidato_munzona_2012_SP.txt
votacao_candidato_munzona_2012_TO.txt


In [263]:
df_votos_12.head()

,DATA_GERACAO,HORA_GERACAO,ANO_ELEICAO,NUM_TURNO (*),DESCRICAO_ELEICAO (*),SIGLA_UF,COD_UE (*),CODIGO_MUNICIPIO (*),NOME_MUNICIPIO,NUMERO_ZONA (*),...,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS
0,28/07/2020,10:42:25,2012,2,ELEIÇÃO MUNICIPAL 2012,AC,1392,1392,RIO BRANCO,9,...,DEFERIDO,1.0,ELEITO,13,PT,PARTIDO DOS TRABALHADORES,10000000002,FRENTE POPULAR DE RIO BRANCO,PRB / PT / PTN / PR / PSDC / PSB / PV / PPL / ...,34292
1,28/07/2020,10:42:25,2012,2,ELEIÇÃO MUNICIPAL 2012,AC,1392,1392,RIO BRANCO,1,...,DEFERIDO,1.0,ELEITO,13,PT,PARTIDO DOS TRABALHADORES,10000000002,FRENTE POPULAR DE RIO BRANCO,PRB / PT / PTN / PR / PSDC / PSB / PV / PPL / ...,27174
2,28/07/2020,10:42:25,2012,1,ELEIÇÃO MUNICIPAL 2012,AC,1570,1570,ASSIS BRASIL,6,...,DEFERIDO,5.0,SUPLENTE,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,10000000177,ALIANÇA RENOVADORA 1,PPS / PSDB / PP,38
3,28/07/2020,10:42:25,2012,1,ELEIÇÃO MUNICIPAL 2012,AC,1570,1570,ASSIS BRASIL,6,...,DEFERIDO,5.0,SUPLENTE,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,10000000177,ALIANÇA RENOVADORA 1,PPS / PSDB / PP,22
4,28/07/2020,10:42:25,2012,1,ELEIÇÃO MUNICIPAL 2012,AC,1570,1570,ASSIS BRASIL,6,...,DEFERIDO,5.0,SUPLENTE,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,10000000177,ALIANÇA RENOVADORA 1,PPS / PSDB / PP,70


In [264]:
df_votos_12.columns

Index(['DATA_GERACAO', 'HORA_GERACAO', 'ANO_ELEICAO', 'NUM_TURNO (*)',
       'DESCRICAO_ELEICAO (*)', 'SIGLA_UF', 'COD_UE (*)',
       'CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'NUMERO_ZONA (*)',
       'CODIGO_CARGO (*)', 'NUMERO_CAND (*)', 'SQ_CANDIDATO (*)',
       'NOME_CANDIDATO', 'NOME_URNA_CANDIDATO', 'DESCRICAO_CARGO',
       'COD_SIT_CAND_SUPERIOR', 'DESC_SIT_CAND_SUPERIOR',
       'CODIGO_SIT_CANDIDATO', 'DESC_SIT_CANDIDATO', 'CODIGO_SIT_CAND_TOT',
       'DESC_SIT_CAND_TOT', 'NUMERO_PARTIDO', 'SIGLA_PARTIDO', 'NOME_PARTIDO',
       'SEQUENCIAL_LEGENDA', 'NOME_COLIGACAO', 'COMPOSICAO_LEGENDA',
       'TOTAL_VOTOS'],
      dtype='object')

In [265]:
set(df_votos_12['DESC_SIT_CAND_TOT'])

{'2º TURNO',
 'ELEITO',
 'ELEITO POR MÉDIA',
 'ELEITO POR QP',
 'NÃO ELEITO',
 'SUPLENTE',
 nan}

In [266]:
set(df_votos_12['DESCRICAO_CARGO'])

{'PREFEITO', 'VEREADOR'}

In [267]:
set(df_votos_12['DESCRICAO_ELEICAO (*)'])

{' ELEIÇÃO SUPLEMENTAR DE MARITUBA/PA',
 ' ELEIÇÃO SUPLEMENTAR DE PRIMAVERA ',
 'ELEICAO SUPLEM. DE BOA VISTA DO GURUPI',
 'ELEIÇÃO MAJORITÁRIA_SERRA DO MEL/RN',
 'ELEIÇÃO MAJ_CAIÇARA DO RIO DO VENTO/RN',
 'ELEIÇÃO MUNICIPAL 2012',
 'ELEIÇÃO SUP. DE CACHOEIRA DOURADA',
 'ELEIÇÃO SUP. DE SANTA MARIA DO PARÁ',
 'ELEIÇÃO SUP. DE SÃO JOÃO DO PARAÍSO',
 'ELEIÇÃO SUPL. DE ÁGUA AZUL DO NORTE',
 'ELEIÇÃO SUPLEM MUQUÉM DO SÃO FRANCISCO',
 'ELEIÇÃO SUPLEM. BREJO DA MADRE DE DEUS',
 'ELEIÇÃO SUPLEMENTAR - CABECEIRAS',
 'ELEIÇÃO SUPLEMENTAR - FLORES DE GOIÁS',
 'ELEIÇÃO SUPLEMENTAR - GOIATUBA',
 'ELEIÇÃO SUPLEMENTAR - NAZÁRIO',
 'ELEIÇÃO SUPLEMENTAR - PIRES DO RIO',
 'ELEIÇÃO SUPLEMENTAR - SÃO DOMINGOS',
 'ELEIÇÃO SUPLEMENTAR AMAPARI-AP',
 'ELEIÇÃO SUPLEMENTAR BARRA DO PIRAÍ-RJ',
 'ELEIÇÃO SUPLEMENTAR CAMAMU-BA',
 'ELEIÇÃO SUPLEMENTAR DE BIQUINHAS',
 'ELEIÇÃO SUPLEMENTAR DE DIAMANTINA',
 'ELEIÇÃO SUPLEMENTAR DE MAJOR ISIDORO',
 'ELEIÇÃO SUPLEMENTAR DE PALESTINA - PA',
 'ELEIÇÃO SUPLEMENTAR DE PALE

In [268]:
len(set(df_votos_12['CODIGO_MUNICIPIO (*)']))

5568

In [269]:
df_prefeituras_12 = df_votos_12[(df_votos_12['DESC_SIT_CAND_TOT'] == 'ELEITO') &\
                                (df_votos_12['DESCRICAO_CARGO'] == 'PREFEITO') &\
                                (df_votos_12['DESCRICAO_ELEICAO (*)'] == 'ELEIÇÃO MUNICIPAL 2012')]


In [270]:
df_prefeituras_resumo_12 = df_prefeituras_12[['CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'SIGLA_UF', 'SIGLA_PARTIDO', 'TOTAL_VOTOS']].groupby(['CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'SIGLA_UF', 'SIGLA_PARTIDO']).sum().reset_index().sort_values('SIGLA_PARTIDO', ascending = False)
df_prefeituras_resumo_12['ano'] = 2012


In [271]:
df_prefeituras_resumo_12.head()

,CODIGO_MUNICIPIO (*),NOME_MUNICIPIO,SIGLA_UF,SIGLA_PARTIDO,TOTAL_VOTOS,ano
439,8397,MONTES ALTOS,MA,PV,2434,2012
937,16144,FERNANDO PEDROZA,RN,PV,1065,2012
919,15954,ITAREMA,CE,PV,12031,2012
4119,78131,RIBEIRÃO CLARO,PR,PV,3788,2012
109,2674,NOVO ARIPUANÃ,AM,PV,2819,2012


In [272]:
len(df_prefeituras_resumo_12)

5563

In [273]:
# Ranking por partido
df_prefeituras_resumo_12[['SIGLA_PARTIDO', 'SIGLA_UF']].groupby(['SIGLA_PARTIDO']).count().reset_index().sort_values('SIGLA_UF', ascending = False).head(10)


,SIGLA_PARTIDO,SIGLA_UF
4,PMDB,1021
16,PSDB,695
20,PT,637
15,PSD,496
6,PP,476
13,PSB,440
2,PDT,307
22,PTB,298
0,DEM,278
9,PR,274


##### -> Validação: todos os municípios estão com resultado? Municípios duplicados?

In [274]:
len(df_prefeituras_resumo_12)

5563

In [275]:
df_teste = df_prefeituras_resumo_12[['CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'SIGLA_UF', 'SIGLA_PARTIDO']].groupby(['CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'SIGLA_UF']).agg({'SIGLA_PARTIDO': pd.Series.nunique})
df_teste[df_teste['SIGLA_PARTIDO'] > 1]


,,,SIGLA_PARTIDO
CODIGO_MUNICIPIO (*),NOME_MUNICIPIO,SIGLA_UF,


In [276]:
df_eleitorado.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,CD_MUN_SIT_BIOMETRIA,DS_MUN_SIT_BIOMETRIA,NR_ZONA,CD_GENERO,...,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
0,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,6,ENSINO MÉDIO COMPLETO,17,1,1,0
1,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,7,SUPERIOR INCOMPLETO,1,0,0,0
2,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9094,90 a 94 anos,8,SUPERIOR COMPLETO,8,0,0,0
3,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9599,95 a 99 anos,1,ANALFABETO,28,0,0,0
4,10/09/2020,23:04:11,2020,SP,71072,SÃO PAULO,0,Sem biometria,257,4,...,3,CASADO,9599,95 a 99 anos,2,LÊ E ESCREVE,149,0,0,0


In [277]:
df_eleitorado.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'CD_MUN_SIT_BIOMETRIA', 'DS_MUN_SIT_BIOMETRIA',
       'NR_ZONA', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'QT_ELEITORES_PERFIL',
       'QT_ELEITORES_BIOMETRIA', 'QT_ELEITORES_DEFICIENCIA',
       'QT_ELEITORES_INC_NM_SOCIAL'],
      dtype='object')

In [278]:
len(set(df_eleitorado['CD_MUNICIPIO']))

5568

In [279]:
df_votos_12.columns

Index(['DATA_GERACAO', 'HORA_GERACAO', 'ANO_ELEICAO', 'NUM_TURNO (*)',
       'DESCRICAO_ELEICAO (*)', 'SIGLA_UF', 'COD_UE (*)',
       'CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'NUMERO_ZONA (*)',
       'CODIGO_CARGO (*)', 'NUMERO_CAND (*)', 'SQ_CANDIDATO (*)',
       'NOME_CANDIDATO', 'NOME_URNA_CANDIDATO', 'DESCRICAO_CARGO',
       'COD_SIT_CAND_SUPERIOR', 'DESC_SIT_CAND_SUPERIOR',
       'CODIGO_SIT_CANDIDATO', 'DESC_SIT_CANDIDATO', 'CODIGO_SIT_CAND_TOT',
       'DESC_SIT_CAND_TOT', 'NUMERO_PARTIDO', 'SIGLA_PARTIDO', 'NOME_PARTIDO',
       'SEQUENCIAL_LEGENDA', 'NOME_COLIGACAO', 'COMPOSICAO_LEGENDA',
       'TOTAL_VOTOS'],
      dtype='object')

In [280]:
df_eleitorado_votos_12 = df_eleitorado[['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'QT_ELEITORES_PERFIL']].groupby(['SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO']).sum().reset_index()\
.merge(df_votos_12[df_votos_12['DESCRICAO_CARGO'] == 'PREFEITO'][['SIGLA_UF', 'CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'DESCRICAO_CARGO', 'TOTAL_VOTOS']].groupby(['SIGLA_UF', 'CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'DESCRICAO_CARGO']).sum(), how = 'left', left_on = ['SG_UF', 'CD_MUNICIPIO'], right_on = ['SIGLA_UF', 'CODIGO_MUNICIPIO (*)'])


In [281]:
df_eleitorado_votos_12.head()

,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,QT_ELEITORES_PERFIL,TOTAL_VOTOS
0,AC,1007,BUJARI,9613,5678
1,AC,1015,CAPIXABA,7540,5782
2,AC,1023,PORTO ACRE,12639,9542
3,AC,1031,SANTA ROSA DO PURUS,3566,2093
4,AC,1040,MARECHAL THAUMATURGO,10491,6720


In [282]:
len(df_eleitorado_votos_12[pd.isna(df_eleitorado_votos_12['TOTAL_VOTOS']) == True])

0

In [283]:
df_prefeituras_resumo_12.columns

Index(['CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'SIGLA_UF', 'SIGLA_PARTIDO',
       'TOTAL_VOTOS', 'ano'],
      dtype='object')

In [284]:
df_prefeituras_resumo_12 = df_prefeituras_resumo_12.merge(tab_municipios_tse[['CD_MUNICIPIO', 'SG_UF', 'CD_MUNICIPIO_IBGE']], how = 'inner', left_on = ['CODIGO_MUNICIPIO (*)', 'SIGLA_UF'], right_on = ['CD_MUNICIPIO', 'SG_UF'])


In [285]:
df_prefeituras_resumo_12.head()

,CODIGO_MUNICIPIO (*),NOME_MUNICIPIO,SIGLA_UF,SIGLA_PARTIDO,TOTAL_VOTOS,ano,CD_MUNICIPIO,SG_UF,CD_MUNICIPIO_IBGE
0,8397,MONTES ALTOS,MA,PV,2434,2012,8397,MA,2107001.0
1,16144,FERNANDO PEDROZA,RN,PV,1065,2012,16144,RN,2403756.0
2,15954,ITAREMA,CE,PV,12031,2012,15954,CE,2306553.0
3,78131,RIBEIRÃO CLARO,PR,PV,3788,2012,78131,PR,4121802.0
4,2674,NOVO ARIPUANÃ,AM,PV,2819,2012,2674,AM,1303304.0


In [286]:
len(df_prefeituras_resumo_12)

5563

In [287]:
df_prefeituras_resumo_12.columns

Index(['CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'SIGLA_UF', 'SIGLA_PARTIDO',
       'TOTAL_VOTOS', 'ano', 'CD_MUNICIPIO', 'SG_UF', 'CD_MUNICIPIO_IBGE'],
      dtype='object')

In [288]:
df_municipios_ibge.columns

Index(['Ano', 'Código da Grande Região', 'Nome da Grande Região',
       'Código da Unidade da Federação', 'Sigla da Unidade da Federação',
       'Nome da Unidade da Federação', 'Código do Município',
       'Nome do Município', 'Região Metropolitana', 'Código da Mesorregião',
       'Nome da Mesorregião', 'Código da Microrregião', 'Nome da Microrregião',
       'Código da Região Geográfica Imediata',
       'Nome da Região Geográfica Imediata',
       'Município da Região Geográfica Imediata',
       'Código da Região Geográfica Intermediária',
       'Nome da Região Geográfica Intermediária',
       'Município da Região Geográfica Intermediária',
       'Código Concentração Urbana', 'Nome Concentração Urbana',
       'Tipo Concentração Urbana', 'Código Arranjo Populacional',
       'Nome Arranjo Populacional', 'Hierarquia Urbana',
       'Hierarquia Urbana (principais categorias)', 'Código da Região Rural',
       'Nome da Região Rural',
       'Região rural (segundo classificação d

In [289]:
df_prefeituras_resumo_12 = df_prefeituras_resumo_12.merge(df_municipios_ibge[df_municipios_ibge['Ano'] == 2018][['Código do Município', 'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)']], how = 'inner', left_on = 'CD_MUNICIPIO_IBGE', right_on = 'Código do Município')


In [290]:
del(df_prefeituras_resumo_12['CD_MUNICIPIO'])
del(df_prefeituras_resumo_12['SG_UF'])
del(df_prefeituras_resumo_12['Código do Município'])


In [291]:
df_prefeituras_resumo_12.columns

Index(['CODIGO_MUNICIPIO (*)', 'NOME_MUNICIPIO', 'SIGLA_UF', 'SIGLA_PARTIDO',
       'TOTAL_VOTOS', 'ano', 'CD_MUNICIPIO_IBGE',
       'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)'],
      dtype='object')

In [292]:
df_prefeituras_resumo_12.columns = ['CD_MUNICIPIO', 'NM_UE', 'SG_UF', 'SG_PARTIDO', 'QT_VOTOS_NOMINAIS', 'ano', 'CD_MUNICIPIO_IBGE', 'PIB a preços correntes (em R$ 1000)']


In [293]:
df_prefeituras_resumo_12

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,PIB a preços correntes (em R$ 1000)
0,8397,MONTES ALTOS,MA,PV,2434,2012,2107001.0,6.610862e+04
1,16144,FERNANDO PEDROZA,RN,PV,1065,2012,2403756.0,3.059930e+04
2,15954,ITAREMA,CE,PV,12031,2012,2306553.0,7.378952e+05
3,78131,RIBEIRÃO CLARO,PR,PV,3788,2012,4121802.0,4.690380e+05
4,2674,NOVO ARIPUANÃ,AM,PV,2819,2012,1303304.0,1.993775e+05
5,7668,GOVERNADOR NUNES FREIRE,MA,PV,6449,2012,2104677.0,2.151179e+05
6,64270,FRANCISCO MORATO,SP,PV,38438,2012,3516309.0,1.517245e+06
7,45675,GUARANI,MG,PV,3092,2012,3128402.0,1.255014e+05
8,7633,CODÓ,MA,PV,23075,2012,2103307.0,9.631802e+05
9,72338,VÁRZEA PAULISTA,SP,PV,29911,2012,3556503.0,2.843701e+06


In [294]:
df_prefeituras_resumo_12 = df_prefeituras_resumo_12.merge(df_eleitorado[['SG_UF', 'CD_MUNICIPIO', 'QT_ELEITORES_PERFIL']].groupby(['SG_UF', 'CD_MUNICIPIO']).sum().reset_index(), how = 'inner', on = ['SG_UF', 'CD_MUNICIPIO'])


In [295]:
df_prefeituras_resumo_16.head()

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,Código do Município,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL
0,46973,JACUTINGA,MG,SD,7753,2016,3134905.0,3134905,692706.027,18297
1,23256,BARRA DE GUABIRABA,PE,SD,4210,2016,2601300.0,2601300,129193.317,9466
2,31399,DIVINA PASTORA,SE,SD,1430,2016,2802007.0,2802007,152679.864,3936
3,31151,BOQUIM,SE,SD,9612,2016,2800670.0,2800670,292363.173,20880
4,13153,ANTONINA DO NORTE,CE,SD,3225,2016,2300804.0,2300804,53422.569,7198


In [296]:
len(df_prefeituras_resumo_16)

5520

In [297]:
print('Conclusão:')
print('Duplicados = ' + str(len(df_teste[df_teste['SIGLA_PARTIDO'] > 1])))
print('Sem prefeito informado = ' + str(len(set(df_eleitorado['CD_MUNICIPIO'])) - len(set(df_prefeituras_resumo_12['CD_MUNICIPIO']))))

Conclusão:
Duplicados = 0
Sem prefeito informado = 5


##### -> Grupos ideológicos

In [298]:
set(df_prefeituras_resumo_12['SG_PARTIDO'])

{'DEM',
 'PC do B',
 'PDT',
 'PHS',
 'PMDB',
 'PMN',
 'PP',
 'PPL',
 'PPS',
 'PR',
 'PRB',
 'PRP',
 'PRTB',
 'PSB',
 'PSC',
 'PSD',
 'PSDB',
 'PSDC',
 'PSL',
 'PSOL',
 'PT',
 'PT do B',
 'PTB',
 'PTC',
 'PTN',
 'PV'}

In [299]:
# Preciso corrigir as siglas, pois há algumas mudanças entre 2012 e 2020.
def corrige_sigla_partido(x):
    if x == 'PMDB':
        y = 'MDB'
    elif x == 'PR':
        y = 'PL'
    elif x == 'PPS':
        y = 'CIDADANIA'
    elif x == 'PRB':
        y = 'REPUBLICANOS'
    elif x == 'SD':
        y = 'SOLIDARIEDADE'
    elif x == 'PHS':
        y = 'PODEMOS'
    elif x == 'PTN':
        y = 'PODEMOS'
    elif x == 'PRP':
        y = 'PATRIOTA'
    elif x == 'PEN':
        y = 'PATRIOTA'
    elif x == 'PT do B':
        y = 'AVANTE'
    elif x == 'PSDC':
        y = 'DC'
    elif x == 'PPL':
        y = 'PCdoB'
    elif x == 'PC do B':
        y = 'PCdoB'
    elif x == 'PATRI':
        y = 'PATRIOTA'
    else:
        y = x
    return y
    

In [300]:
df_prefeituras_resumo_12_ = df_prefeituras_resumo_12.copy()
df_prefeituras_resumo_12_['SG_PARTIDO'] = df_prefeituras_resumo_12_['SG_PARTIDO'].map(corrige_sigla_partido)
df_prefeituras_resumo_12_ = df_prefeituras_resumo_12_.merge(df_resumo_posicionamentos_[['Partido', 'grupo_ideologico']], how = 'inner', left_on = 'SG_PARTIDO', right_on = 'Partido')


In [301]:
df_prefeituras_resumo_12_

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL,Partido,grupo_ideologico
0,8397,MONTES ALTOS,MA,PV,2434,2012,2107001.0,6.610862e+04,6733,PV,liberal_progressista
1,16144,FERNANDO PEDROZA,RN,PV,1065,2012,2403756.0,3.059930e+04,3075,PV,liberal_progressista
2,15954,ITAREMA,CE,PV,12031,2012,2306553.0,7.378952e+05,30050,PV,liberal_progressista
3,78131,RIBEIRÃO CLARO,PR,PV,3788,2012,4121802.0,4.690380e+05,9603,PV,liberal_progressista
4,2674,NOVO ARIPUANÃ,AM,PV,2819,2012,1303304.0,1.993775e+05,13070,PV,liberal_progressista
5,7668,GOVERNADOR NUNES FREIRE,MA,PV,6449,2012,2104677.0,2.151179e+05,17641,PV,liberal_progressista
6,64270,FRANCISCO MORATO,SP,PV,38438,2012,3516309.0,1.517245e+06,126392,PV,liberal_progressista
7,45675,GUARANI,MG,PV,3092,2012,3128402.0,1.255014e+05,6990,PV,liberal_progressista
8,7633,CODÓ,MA,PV,23075,2012,2103307.0,9.631802e+05,72921,PV,liberal_progressista
9,72338,VÁRZEA PAULISTA,SP,PV,29911,2012,3556503.0,2.843701e+06,79011,PV,liberal_progressista


In [302]:
df_prefeituras_resumo_12_['NM_UE_UF'] = df_prefeituras_resumo_12_['NM_UE'] + '-' + df_prefeituras_resumo_12_['SG_UF']

In [303]:
df_prefeituras_resumo_12_[['NM_UE_UF', 'grupo_ideologico', 'SG_PARTIDO', 'QT_ELEITORES_PERFIL', 'PIB a preços correntes (em R$ 1000)']].sample(5)


,NM_UE_UF,grupo_ideologico,SG_PARTIDO,QT_ELEITORES_PERFIL,PIB a preços correntes (em R$ 1000)
1465,JUNQUEIRÓPOLIS-SP,liberal_progressista,PSDB,13671,576140.896
3651,JURAMENTO-MG,liberal_conservador,PP,4461,38620.681
3698,CACHOEIRA-BA,liberal_conservador,PP,25606,473573.597
959,AMETISTA DO SUL-RS,intervencionista_progressista,PT,5585,120279.375
4134,BOM JARDIM DE MINAS-MG,liberal_conservador,MDB,5770,108238.012


In [304]:
len(df_prefeituras_resumo_12_)

5475

In [305]:
df_prefeituras_resumo_12_[['SG_PARTIDO', 'grupo_ideologico']].drop_duplicates().sort_values('grupo_ideologico')

,SG_PARTIDO,grupo_ideologico
3328,PCdoB,intervencionista_progressista
451,PT,intervencionista_progressista
1088,PSOL,intervencionista_progressista
4890,PDT,intervencionista_progressista
2304,PSC,liberal_conservador
3869,MDB,liberal_conservador
3393,PP,liberal_conservador
3203,CIDADANIA,liberal_conservador
2929,PL,liberal_conservador
2849,REPUBLICANOS,liberal_conservador


In [306]:
df_prefeituras_resumo_12_[['NM_UE_UF', 'grupo_ideologico', 'SG_PARTIDO', 'QT_ELEITORES_PERFIL', 'PIB a preços correntes (em R$ 1000)']]\
.to_excel('df_prefeituras_resumo_12.xlsx', encoding = 'latin1')


### 4. Analisando junto

In [307]:
df_prefeituras_resumo_12_.head(3)

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL,Partido,grupo_ideologico,NM_UE_UF
0,8397,MONTES ALTOS,MA,PV,2434,2012,2107001.0,66108.625,6733,PV,liberal_progressista,MONTES ALTOS-MA
1,16144,FERNANDO PEDROZA,RN,PV,1065,2012,2403756.0,30599.300,3075,PV,liberal_progressista,FERNANDO PEDROZA-RN
2,15954,ITAREMA,CE,PV,12031,2012,2306553.0,737895.246,30050,PV,liberal_progressista,ITAREMA-CE


In [308]:
df_prefeituras_resumo_16_.head(3)

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,Código do Município,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL,Partido,grupo_ideologico,NM_UE_UF
0,46973,JACUTINGA,MG,SOLIDARIEDADE,7753,2016,3134905.0,3134905,692706.027,18297,SOLIDARIEDADE,liberal_conservador,JACUTINGA-MG
1,23256,BARRA DE GUABIRABA,PE,SOLIDARIEDADE,4210,2016,2601300.0,2601300,129193.317,9466,SOLIDARIEDADE,liberal_conservador,BARRA DE GUABIRABA-PE
2,31399,DIVINA PASTORA,SE,SOLIDARIEDADE,1430,2016,2802007.0,2802007,152679.864,3936,SOLIDARIEDADE,liberal_conservador,DIVINA PASTORA-SE


In [309]:
df_prefeituras_resumo_20_.head(3)

,CD_MUNICIPIO,NM_UE,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS,ano,CD_MUNICIPIO_IBGE,Código do Município,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL,Partido,grupo_ideologico,NM_UE_UF
0,4766,CACHOEIRA DO PIRIÁ,PA,SOLIDARIEDADE,6121,2020,1501956.0,1501956,179439.409,14219,SOLIDARIEDADE,liberal_conservador,CACHOEIRA DO PIRIÁ-PA
1,96938,ÁGUA FRIA DE GOIÁS,GO,SOLIDARIEDADE,1820,2020,5200175.0,5200175,255565.620,4586,SOLIDARIEDADE,liberal_conservador,ÁGUA FRIA DE GOIÁS-GO
2,89826,NOVA SANTA HELENA,MT,SOLIDARIEDADE,1291,2020,5106190.0,5106190,174711.050,2828,SOLIDARIEDADE,liberal_conservador,NOVA SANTA HELENA-MT


In [310]:
df_resumo_grupos = pd.concat([df_prefeituras_resumo_12_[['ano', 'grupo_ideologico', 'PIB a preços correntes (em R$ 1000)', 'QT_ELEITORES_PERFIL']].groupby(['ano', 'grupo_ideologico']).sum().reset_index(),\
                              df_prefeituras_resumo_16_[['ano', 'grupo_ideologico', 'PIB a preços correntes (em R$ 1000)', 'QT_ELEITORES_PERFIL']].groupby(['ano', 'grupo_ideologico']).sum().reset_index(),\
                              df_prefeituras_resumo_20_[['ano', 'grupo_ideologico', 'PIB a preços correntes (em R$ 1000)', 'QT_ELEITORES_PERFIL']].groupby(['ano', 'grupo_ideologico']).sum().reset_index()])



In [311]:
df_resumo_grupos

,ano,grupo_ideologico,PIB a preços correntes (em R$ 1000),QT_ELEITORES_PERFIL
0,2012,intervencionista_progressista,2.278814e+09,41441754
1,2012,liberal_conservador,2.787038e+09,66238683
2,2012,liberal_progressista,1.617212e+09,38240134
0,2016,intervencionista_progressista,5.687879e+08,16013881
1,2016,liberal_conservador,3.335780e+09,77767362
2,2016,liberal_progressista,2.663992e+09,50432109
0,2020,intervencionista_progressista,5.098600e+08,13944935
1,2020,liberal_conservador,4.351299e+09,99104548
2,2020,liberal_progressista,1.800824e+09,32727788


In [317]:
ano = 2020
medida = 'PIB a preços correntes (em R$ 1000)'

df_proporcao = df_resumo_grupos[df_resumo_grupos['ano'] == ano][['grupo_ideologico', medida]]
df_proporcao['perc'] = df_proporcao[medida].apply(lambda x: x/df_proporcao[medida].sum())
df_proporcao

,grupo_ideologico,PIB a preços correntes (em R$ 1000),perc
0,intervencionista_progressista,5.098600e+08,0.076533
1,liberal_conservador,4.351299e+09,0.653154
2,liberal_progressista,1.800824e+09,0.270313


In [320]:
ano = 2020
medida = 'QT_ELEITORES_PERFIL'

df_proporcao = df_resumo_grupos[df_resumo_grupos['ano'] == ano][['grupo_ideologico', medida]]
df_proporcao['perc'] = df_proporcao[medida].apply(lambda x: x/df_proporcao[medida].sum())
df_proporcao

,grupo_ideologico,QT_ELEITORES_PERFIL,perc
0,intervencionista_progressista,13944935,0.095659
1,liberal_conservador,99104548,0.679835
2,liberal_progressista,32727788,0.224505
